# 完全弹性碰撞｜仅包含滑动摩擦力｜完全散热的独立系统｜质量相等的运动点模拟

In [1]:
import numpy as np
import drawSvg
from tqdm import tqdm
import os

In [2]:
def pointDraw(points,r:int=10,frameSize:tuple=(1786,832)):
    """点画，目前仅支持最大12个点"""
    w,h = frameSize

    color = ['grey','SeaGreen' ,'grey','SteelBlue','grey','Orange','grey','LightCoral','grey','DarkSlateBlue','grey','CornflowerBlue']
    points = np.array(points)
    img = drawSvg.Drawing(w,h, displayInline=False)
    rec = drawSvg.Rectangle(0,0,w,h, fill='white')
    img.append(rec)

    for i in range(points.shape[0]):
        X,Y = points[i]
        c = color[i]
        img.append(drawSvg.Circle(X,Y, r,fill=c, stroke_width=1, stroke=c, fill_opacity=0.6))
    img.setPixelScale(2)
    img.rasterize()
    return img

In [3]:
def getInitLocation(pNum,r,frameSize:tuple=(1786,832)):
    """随机初始位置"""
    w,h = frameSize
    initLocation = np.array([np.random.randint(r*2,w-r*2,pNum),np.random.randint(r*2,h-r*2,pNum)]).T
    return initLocation

In [4]:
def getInitSpeed(pNum,low,high,ifEndToEnd = False):
    """随机初始速度"""
    initSpeed = np.array([np.random.randint(low,high,pNum),np.random.randint(low,high,pNum)]).T
    if ifEndToEnd == False:
        ones = np.random.randint(-1,2,300)
        ones = ones[np.where(ones!=0)]
        Xones = ones[:pNum]
        Yones = ones[-pNum:]
    else:
        ones = np.random.randint(-1,2,300)
        ones = ones[np.where(ones!=0)]
        Xones = np.ones(pNum)
        Yones = ones[-pNum:]
    initSpeed[:,0] = initSpeed[:,0]*Xones
    initSpeed[:,1] = initSpeed[:,1]*Yones
    return initSpeed

In [5]:
def calEK(m,speedList):
    """计算动能
    
    ek = (mv^2)/2
    """
    pNum = speedList.shape[0]
    mList = np.array([m for i in range(pNum)])
    sl2 = speedList[:,0]**2 + speedList[:,1]**2
    ekList = 0.5*mList*sl2
    return ekList

In [6]:
def calStopTime(m,miu,pNum,gravity,speedList):
    """在一个完全弹性碰撞、散热良好的封闭系统中，初始动能完全被摩擦力做功消耗；
    
    因此初始动能全部转换为热: Ek = miu*m*g*distance
    
    假设速度只受到滑动摩擦力做功，从整体上看是一个速度衰减的多元线性系统，因此其平均速度为avg(v_start,v_end)，基于此计算直至静止的时间"""
    d = np.sum(calEK(m,speedList))/(miu*m*gravity*pNum)
    v = np.sum(np.sqrt(speedList[:,0]**2 + speedList[:,1]**2))
    t = d/(v/2)
    return t

In [7]:
def updateLocation(currentSpeed,currentLocation,acc,t):
    """按照匀减速直线运动更新位置，要注意acc是负的"""
    currentLocation = np.array(currentLocation)
    currentSpeed = np.array(currentSpeed)
    deltaLocation = t*currentSpeed + (acc*(t**2))/2
    presentLocation = currentLocation + deltaLocation
    return presentLocation

In [8]:
def updateSpeed(currentSpeed,acc,t):
    """按照匀减速直线运动更新速度，要注意acc是负的"""
    deltaSpeed = acc*t
    absSpeed = np.abs(currentSpeed)
    param = currentSpeed/absSpeed
    param = np.nan_to_num(param)
    presentSpeed = absSpeed + deltaSpeed
    presentSpeed = param*presentSpeed
    return presentSpeed

In [9]:
def calDis(currentLocation):
    """计算点与点的距离"""
    Xd = currentLocation.T[0,:]-currentLocation[:,0].reshape(-1,1)
    Yd = currentLocation.T[1,:]-currentLocation[:,1].reshape(-1,1)
    dis = np.sqrt(Xd**2 + Yd**2)
    return dis

In [10]:
def checkClps(dis,r):
    """检测点与点的碰撞，返回有碰撞的点组索引"""
    index = np.array(np.where(dis<=2*r)).T
    check = index[:,0] - index[:,1]
    index = index[np.where(check!=0)]
    l = []
    for i in index.tolist():
        x,y = i
        if i not in l and [y,x] not in l:
            l.append(i)
    l = np.array(l)
    return l

In [11]:
def checkBorder(currentLocation,r,frameSize:tuple=(1786,832)):
    """检测点与边界线的碰撞，返回有碰撞的点"""
    w,h = frameSize
    checkX0 = np.where(currentLocation[:,0]-r<=0)[0]
    checkY0 = np.where(currentLocation[:,1]-r<=0)[0]
    checkXb = np.where(currentLocation[:,0]+r>=w)[0]
    checkYb = np.where(currentLocation[:,1]+r>=h)[0]
    return {'X0':checkX0,'Y0':checkY0,'Xb':checkXb,'Yb':checkYb}

In [12]:
def lastswitchPair(currentSpeed,clpsIndex,lastClpsIndex):
    """完全弹性碰撞的点对直接换向"""
    if clpsIndex.shape[0] == 0:
        return currentSpeed
    for pair in clpsIndex:
        if pair not in lastClpsIndex:
            a,b = pair
            vax,vay = currentSpeed[a]
            vbx,vby = currentSpeed[b]
            currentSpeed[a] = vbx,vby
            currentSpeed[b] = vax,vay
    return currentSpeed

In [13]:
def switchPair(currentSpeed,clpsIndex):
    """完全弹性碰撞的点对直接换向"""
    if clpsIndex.shape[0] == 0:
        return currentSpeed
    for pair in clpsIndex:
        a,b = pair
        vax,vay = currentSpeed[a]
        vbx,vby = currentSpeed[b]
        currentSpeed[a] = vbx,vby
        currentSpeed[b] = vax,vay
    return currentSpeed

In [14]:
def lastswitchBorder(currentSpeed,bdIndex,lastBorderIndex):
    """完全弹性碰撞的点与边界单个反向"""
    X0 = bdIndex['X0']
    Y0 = bdIndex['Y0']
    Xb = bdIndex['Xb']
    Yb = bdIndex['Yb']
    lX0 = lastBorderIndex['X0']
    lY0 = lastBorderIndex['Y0']
    lXb = lastBorderIndex['Xb']
    lYb = lastBorderIndex['Yb']
    if X0.shape[0] == 0:
        pass
    else:
        for point in X0:
            if point not in lX0:
                currentSpeed[point,0] = -currentSpeed[point,0]

    if Y0.shape[0] == 0:
        pass
    else:
        for point in Y0:
            if point not in lY0:
                currentSpeed[point,1] = -currentSpeed[point,1] 

    if Xb.shape[0] == 0:
        pass
    else:
        for point in Xb:
            if point not in lXb:
                currentSpeed[point,0] = -currentSpeed[point,0]

    if Yb.shape[0] == 0:
        pass
    else:
        for point in Yb:
            if point not in lYb:
                currentSpeed[point,1] = -currentSpeed[point,1]
            
    return currentSpeed

In [15]:
def switchBorder(currentSpeed,bdIndex):
    """完全弹性碰撞的点与边界单个反向"""
    X0 = bdIndex['X0']
    Y0 = bdIndex['Y0']
    Xb = bdIndex['Xb']
    Yb = bdIndex['Yb']
    if X0.shape[0] == 0:
        pass
    else:
        for point in X0:
            currentSpeed[point,0] = -currentSpeed[point,0]

    if Y0.shape[0] == 0:
        pass
    else:
        for point in Y0:
            currentSpeed[point,1] = -currentSpeed[point,1] 

    if Xb.shape[0] == 0:
        pass
    else:
        for point in Xb:
            currentSpeed[point,0] = -currentSpeed[point,0]

    if Yb.shape[0] == 0:
        pass
    else:
        for point in Yb:
            currentSpeed[point,1] = -currentSpeed[point,1]
            
    return currentSpeed

In [16]:
def lastmoveItAndSave(pNum,r,initLocation,initSpeed,miu,m,gravity,calcT,path,maxIter=600):
    currentLocation = initLocation.copy()
    currentSpeed = initSpeed.copy()
    ct = 0
    pbar = tqdm(total=maxIter)
    pbar.set_description('Processing:')
    loc = []
    while ct <= maxIter:
        if len(loc) <=1:
            lastLocation = []
        else:
            lastLocation = np.array(loc[-2])
        # 点间距
        dis = calDis(currentLocation)
        if len(lastLocation) != 0:
            lastDis = calDis(lastLocation)
        # 点是否碰撞
        clps = checkClps(dis,r)
        if len(lastLocation) != 0:
            lastClps = checkClps(lastDis,r)
        else:
            lastClps = []
        # 与边境是否碰撞
        border = checkBorder(currentLocation,r)
        if len(lastLocation) != 0:
            lastBorder = checkBorder(lastLocation,r)
        else:
            lastBorder  = {'X0':[],'Y0':[],'Xb':[],'Yb':[]}
        # 更新点碰撞速度  
        currentSpeed = switchPair(currentSpeed,clps,lastClps)
        # 更新边缘碰撞速度
        currentSpeed = switchBorder(currentSpeed,border,lastBorder)
        # 计算新的位置
        # print(currentSpeed,currentLocation,-miu*gravity,t)
        currentLocation = updateLocation(currentSpeed= currentSpeed,currentLocation= currentLocation,acc= -miu*gravity,t=calcT)
        # 计算新的速度
        # currentSpeed = updateSpeed(currentSpeed=currentSpeed,acc=-miu*gravity,t=calcT)
        # 画点
        img = pointDraw(currentLocation,r)
        # 存点
        savePath = path+'frame'+str(ct) +".png"
        img.savePng(savePath)
        loc.append(currentLocation.tolist())
        ct+=1
        pbar.update(1)
    return loc

In [17]:
def moveItAndSave(pNum,r,initLocation,initSpeed,miu,m,gravity,calcT,path,maxIter=600):
    currentLocation = initLocation.copy()
    currentSpeed = initSpeed.copy()
    ct = 0
    pbar = tqdm(total=maxIter)
    pbar.set_description('Processing:')
    loc = []
    while ct <= maxIter:
        # 点间距
        dis = calDis(currentLocation)
        # 点是否碰撞
        clps = checkClps(dis,r)
        # 与边境是否碰撞
        border = checkBorder(currentLocation,r)
        # 更新点碰撞速度  
        currentSpeed = switchPair(currentSpeed,clps)
        # 更新边缘碰撞速度
        currentSpeed = switchBorder(currentSpeed,border)
        # 计算新的位置
        # print(currentSpeed,currentLocation,-miu*gravity,t)
        currentLocation = updateLocation(currentSpeed= currentSpeed,currentLocation= currentLocation,acc= -miu*gravity,t=calcT)
        # 计算新的速度
        # currentSpeed = updateSpeed(currentSpeed=currentSpeed,acc=-miu*gravity,t=calcT)
        # 画点
        img = pointDraw(currentLocation,r)
        # 存点
        savePath = path+'frame'+str(ct) +".png"
        img.savePng(savePath)
        loc.append(currentLocation.tolist())
        ct+=1
        pbar.update(1)
    return loc

In [18]:
def lastjustMoveIt(pNum,r,initLocation,initSpeed,miu,m,gravity,calcT,maxIter=600):
    currentLocation = initLocation.copy()
    currentSpeed = initSpeed.copy()
    ct = 0
    pbar = tqdm(total=maxIter)
    loc = []
    pbar.set_description('Processing :')
    while ct <= maxIter:
        if len(loc) <=1:
            lastLocation = []
        else:
            lastLocation = np.array(loc[-2])
        # 点间距
        dis = calDis(currentLocation)
        if len(lastLocation) != 0:
            lastDis = calDis(lastLocation)
        # 点是否碰撞
        clps = checkClps(dis,r)
        if len(lastLocation) != 0:
            lastClps = checkClps(lastDis,r)
        else:
            lastClps = []
        # 与边境是否碰撞
        border = checkBorder(currentLocation,r)
        if len(lastLocation) != 0:
            lastBorder = checkBorder(lastLocation,r)
        else:
            lastBorder  = {'X0':np.array([]),'Y0':np.array([]),'Xb':np.array([]),'Yb':np.array([])}
        # 更新点碰撞速度  
        currentSpeed = lastswitchPair(currentSpeed,clps,lastClps)
        # 更新边缘碰撞速度
        currentSpeed = lastswitchBorder(currentSpeed,border,lastBorder)
        # 计算新的位置
        # print(currentSpeed,currentLocation,-miu*gravity,t)
        currentLocation = updateLocation(currentSpeed= currentSpeed,currentLocation= currentLocation,acc= -miu*gravity,t=calcT)
        # 计算新的速度
        # currentSpeed = updateSpeed(currentSpeed=currentSpeed,acc=-miu*gravity,t=calcT)
        ct+=1
        loc.append(currentLocation.tolist())
        pbar.update(1)
    return loc

In [19]:
def justMoveIt(pNum,r,initLocation,initSpeed,miu,m,gravity,calcT,maxIter=600):
    currentLocation = initLocation.copy()
    currentSpeed = initSpeed.copy()
    ct = 0
    pbar = tqdm(total=maxIter)
    loc = []
    pbar.set_description('Processing :')
    while ct <= maxIter:
        # 点间距
        dis = calDis(currentLocation)
        # 点是否碰撞
        clps = checkClps(dis,r)
        # 与边境是否碰撞
        border = checkBorder(currentLocation,r)
        # 更新点碰撞速度  
        currentSpeed = switchPair(currentSpeed,clps)
        # 更新边缘碰撞速度
        currentSpeed = switchBorder(currentSpeed,border)
        # 计算新的位置
        # print(currentSpeed,currentLocation,-miu*gravity,t)
        currentLocation = updateLocation(currentSpeed= currentSpeed,currentLocation= currentLocation,acc= -miu*gravity,t=calcT)
        # 计算新的速度
        # currentSpeed = updateSpeed(currentSpeed=currentSpeed,acc=-miu*gravity,t=calcT)
        ct+=1
        loc.append(currentLocation.tolist())
        pbar.update(1)
    return loc

In [20]:
def genGIF(path,name,rate=25):
    """存储img为gif，时间很久，平均0.2s一个frame"""
    sh = """
    cd {path}
    ffmpeg -r {rate} -i frame%d.png mot.avi
    ffmpeg -i mot.avi -pix_fmt rgb24 {name}
    """
    rate = str(rate)
    os.system(sh.format(path=path,rate=rate,name=name))

In [21]:
FRAMESIZE = (1786,832)
g = 10
M = 1.0
MIU = 0.2
RADIUS = 20
MIN_SPEED = 60
MAX_SPEED = 80
FRAMERATE = 25
CALCTIME = 1/FRAMERATE

In [22]:
START_BORDER = FRAMESIZE[0]//7,FRAMESIZE[1]//7
START_BORDER

(255, 118)

In [261]:
pNum = 12
w,h = FRAMESIZE
initLocation = np.array([np.random.randint(RADIUS*2,START_BORDER[0],pNum),np.random.randint(RADIUS*2,h-RADIUS*2,pNum)]).T
img = pointDraw(initLocation,RADIUS)
img

In [263]:
initSpeed = getInitSpeed(pNum,MIN_SPEED,MAX_SPEED,True)
initSpeed[:,1] = initSpeed[:,1]
initSpeed

array([[ 65, -82],
       [ 86, -84],
       [ 60, -79],
       [ 85,  87],
       [ 76, -60],
       [ 79,  80],
       [ 67, -78],
       [ 76, -76],
       [ 71,  76],
       [ 76,  84],
       [ 88,  88],
       [ 87,  66]])

In [264]:
loc = justMoveIt(pNum=pNum,r=RADIUS,initLocation=initLocation,initSpeed=initSpeed,miu=MIU,m=M,gravity=g,calcT=CALCTIME,maxIter=550)

Processing :: : 551it [00:00, 849.04it/s]                        


In [265]:
img = pointDraw(loc[-1],RADIUS)
img

In [25]:
i = 0
pbar = tqdm(total = len(loc))
for p in loc:
    img = pointDraw(p,RADIUS)
    savePath = './motImg03122/'+'frame'+str(i) +".png"
    img.savePng(savePath)
    i += 1
    pbar.update(1)

100%|██████████| 451/451 [03:31<00:00,  2.26it/s]

In [28]:
moveItAndSave(pNum=pNum,r=RADIUS,initLocation=initLocation,initSpeed=initSpeed,miu=MIU,m=M,gravity=g,calcT=CALCTIME,path = './mot0424/',maxIter=500)

Processing:: : 501it [03:35,  2.32it/s]                       


In [29]:
genGIF('./mot0424/','mot.gif',60)

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with Apple clang version 12.0.0 (clang-1200.0.32.28)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.3.1_8 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librtmp --enable-libspeex --enable-libsoxr --enable-videotoolbox --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=j

# 0426 重新生成

In [23]:
import json
import os

In [24]:
def pointDis(data:np.array):
    length = data.shape[0]
    out = []
    for i in range(length+1):
        for j in range(i+1,length):
            minus = data[i]-data[j]
            sqrtMinus = minus**2
            final = np.sqrt(np.sum(sqrtMinus))
            out.append(final)
    return np.array(out)

In [25]:
def dis(A:np.array,B:np.array):
    return np.sqrt((A[:,0] - B[:,0])**2 + (A[:,1] -B[:,1])**2)

In [26]:
def multiDis(data:np.array):
    length = data.shape[0]
    l = []
    for i in range(length):
        for j in range(i+1,length):
            l.append(dis(data[i],data[j]))
    return np.array(l).T

In [27]:
def startPoints(pNum,a=2,b=3):
    """轮胎算法生成初始点位置"""
    r = np.arange(pNum+1)
    a1, a2 = np.sqrt(a), np.sqrt(b)
    x1, x2 = r*a1, r*a2
    y1, y2 = np.floor(x1), np.floor(x2)
    x1, x2 = x1-y1, x2-y2
    return x1[1:],x2[1:]

In [28]:
with open("8-1-1-17.json","r") as f:
    sample = json.loads(f.read())
    

In [35]:
START_BORDER = FRAMESIZE[0]//7,FRAMESIZE[1]

In [69]:
with open("./mot0507-ori/design.json","r") as f:
    design = json.loads(f.read())

In [64]:
def generate(config:list,times:int,path:str,sep = 7):
    """集合函数，通过design生成json"""
    global RADIUS, MIU, M, g, CALCTIME,sample
    START_BORDER = FRAMESIZE[0]//sep,FRAMESIZE[1]
    stage = config[0]
    pNum = config[1]
    tgt = config[2]
    xvRate = config[3]
    yvRate = config[4]
    theRightAnswer = config[5]
    targetPlaneIndex = config[6]
    
    s_pNum = "0" + str(pNum) if pNum<10 else str(pNum)

    itemId = stage + s_pNum + "0" + str(tgt) + str(xvRate)+ str(yvRate) +"0" + str(sep) + "-" +str(times)
    x,y = startPoints(pNum,a=10,b=14)
    initLocation = np.array([START_BORDER[0]*x, START_BORDER[1]*y]).T

    initSpeed = getInitSpeed(pNum,MIN_SPEED,MAX_SPEED,True)
    initSpeed = np.abs(initSpeed)
    initSpeed[:,0] = initSpeed[:,0]*xvRate/10
    initSpeed[:,1] = initSpeed[:,1]*yvRate/10

    # 四种对半分的y速度值,现在的是[正n/2，负n/2]
    if times in [1,5,9]:
        yMovement = [1 for _ in range(int(initSpeed.shape[0]/2))] + [-1 for _ in range(int(initSpeed.shape[0]/2))]
    elif times in [2,6,10]:
        yMovement = [-1 for _ in range(int(initSpeed.shape[0]/2))] + [1 for _ in range(int(initSpeed.shape[0]/2))]
    elif times in [3,7,11]:
        yMovement = [1 if float(i//2)==i/2 else -1 for i in range(initSpeed.shape[0])]
    elif times in [4,8,12]:
        yMovement = [-1 if float(i//2)==i/2 else 1 for i in range(initSpeed.shape[0])]

    yMovement = yMovement+[-1 for _ in range(initSpeed.shape[0]-len(yMovement))] if initSpeed.shape[0]-len(yMovement) != 0 else yMovement 

    initSpeed[:,1] = initSpeed[:,1]*yMovement
    loc = justMoveIt(pNum=pNum,r=RADIUS,initLocation=initLocation,initSpeed=initSpeed,miu=MIU,m=M,gravity=g,calcT=CALCTIME,maxIter=500)
    l = []
    for i in range(pNum):
        l.append(np.array(loc)[:,i,:].tolist())
    l = np.round(np.array(l),3).tolist()
    sample['ctrl'] = l
    sample['itemID'] = itemId
    sample['hardLevel'] = itemId[:-2]
    sample['planeTargetNum'] = tgt
    sample["theRightAnswer"] = theRightAnswer
    sample["targetPlaneIndex"] = targetPlaneIndex
    with open(path+itemId+".json","w+") as f:
        f.write(json.dumps(sample))

In [65]:
for config in design:
    if config[0] == "lx":
        for i in range(2):
            generate(config,i+1,"./mot0507/",sep = 7)
    else:
        for i in range(3):
            generate(config,i+1,"./mot0507/",sep = 7)

Processing :: : 501it [00:00, 12049.69it/s]          
Processing :: : 501it [00:00, 15696.92it/s]          
Processing :: : 501it [00:00, 15905.67it/s]          
Processing :: : 501it [00:00, 16490.07it/s]          
Processing :: : 501it [00:00, 15977.88it/s]          
Processing :: : 501it [00:00, 17708.82it/s]          
Processing :: : 501it [00:00, 13334.35it/s]          
Processing :: : 501it [00:00, 15194.23it/s]          
Processing :: : 501it [00:00, 15741.37it/s]          
Processing :: : 501it [00:00, 12138.44it/s]          
Processing :: : 501it [00:00, 15974.72it/s]          
Processing :: : 501it [00:00, 15314.04it/s]          
Processing :: : 501it [00:00, 17149.37it/s]          
Processing :: : 501it [00:00, 11516.18it/s]          
Processing :: : 501it [00:00, 15988.57it/s]          
Processing :: : 501it [00:00, 13453.27it/s]          
Processing :: : 501it [00:00, 13656.46it/s]          
Processing :: : 501it [00:00, 18201.04it/s]          
Processing :: : 501it [00:00

In [79]:
design[-6],design[-7]

(['fm', 12, 4, 10, 20, '2-4-6-8', [1, 3, 5, 7]],
 ['fm', 12, 4, 10, 15, '2-4-6-8', [1, 3, 5, 7]])

In [81]:
generate(design[-7],1,"./mot0507-last/",sep = 7)

Processing :: : 501it [00:00, 13263.14it/s]          


In [51]:
# pNum = 3
# xvRate = 10
# yvRate = 20
# tgt = 2
# itemId = "lx1206102007-12"
# x,y = startPoints(pNum,a=10,b=14)
# initLocation = np.array([START_BORDER[0]*x, START_BORDER[1]*y]).T

# initSpeed = getInitSpeed(pNum,MIN_SPEED,MAX_SPEED,True)
# initSpeed = np.abs(initSpeed)
# initSpeed[:,0] = initSpeed[:,0]*xvRate/10
# initSpeed[:,1] = initSpeed[:,1]*yvRate/10

# # 四种对半分的y速度值,现在的是[正n/2，负n/2]
# yMovement = [1 for _ in range(int(initSpeed.shape[0]/2))] + [-1 for _ in range(int(initSpeed.shape[0]/2))]
# # yMovement = [-1 for _ in range(int(initSpeed.shape[0]/2))] + [1 for _ in range(int(initSpeed.shape[0]/2))]
# # yMovement = [1 if float(i//2)==i/2 else -1 for i in range(initSpeed.shape[0])]
# # yMovement = [-1 if float(i//2)==i/2 else 1 for i in range(initSpeed.shape[0])]

# yMovement = yMovement+[-1 for _ in range(initSpeed.shape[0]-len(yMovement))] if initSpeed.shape[0]-len(yMovement) != 0 else yMovement 

# initSpeed[:,1] = initSpeed[:,1]*yMovement
# initSpeed 

array([[  77,  126],
       [  73, -138],
       [  66, -148]])

In [34]:
# loc = justMoveIt(pNum=pNum,r=RADIUS,initLocation=initLocation,initSpeed=initSpeed,miu=MIU,m=M,gravity=g,calcT=CALCTIME,maxIter=500)
# l = []
# for i in range(pNum):
#     l.append(np.array(loc)[:,i,:].tolist())
# l = np.round(np.array(l),3).tolist()
# sample['ctrl'] = l
# sample['itemID'] = itemId
# sample['hardLevel'] = itemId[:-3]
# sample['planeTargetNum'] = tgt
# with open("./mot0507/"+itemId+".json","a") as f:
#     f.write(json.dumps(sample))

Processing :: : 501it [00:00, 12367.04it/s]          


In [88]:
fList = os.listdir("./mot0507-last")
fList = [i for i in fList if "._" not in i]
fList = [i for i in fList if ".DS" not in i]
fList = [i for i in fList if "design" not in i]
fList.sort()
fList = fList[-4:] + [i for i in fList[:-4] if "1010" in i]+ [i for i in fList[:-4] if "1015" in i]+ [i for i in fList[:-4] if "1020" in i]+ [i for i in fList[:-4] if "2010" in i]
fList

['lx0201101007-1.json',
 'lx0201101007-2.json',
 'lx0302101007-1.json',
 'lx0302101007-2.json',
 'fm0402101007-1.json',
 'fm0402101007-2.json',
 'fm0603101007-1.json',
 'fm0603101007-2.json',
 'fm0603101007-3.json',
 'fm0804101007-1.json',
 'fm0804101007-2.json',
 'fm0804101007-3.json',
 'fm1004101007-1.json',
 'fm1004101007-2.json',
 'fm1004101007-3.json',
 'fm1204101007-1.json',
 'fm1204101007-2.json',
 'fm1204101007-3.json',
 'fm1206101007-1.json',
 'fm1206101007-2.json',
 'fm1206101007-3.json',
 'fm0402101507-1.json',
 'fm0402101507-2.json',
 'fm0603101507-1.json',
 'fm0603101507-2.json',
 'fm0603101507-3.json',
 'fm0804101507-1.json',
 'fm0804101507-2.json',
 'fm0804101507-3.json',
 'fm1004101507-1.json',
 'fm1004101507-2.json',
 'fm1004101507-3.json',
 'fm1204101507-1.json',
 'fm1204101507-2.json',
 'fm1204101507-3.json',
 'fm1206101507-1.json',
 'fm1206101507-2.json',
 'fm1206101507-3.json',
 'fm0402102007-1.json',
 'fm0402102007-2.json',
 'fm0603102007-1.json',
 'fm0603102007-2

In [89]:
l = []
for f in fList:
    try:
        with open("./mot0507-last/"+f ,"r") as f:
            l.append(json.loads(f.read()))
    except Exception as e:
        print(f)
with open("./mot0507-last/comb.json" ,"w+") as f:
        f.write(json.dumps(l))

In [7]:
with open("./mot0506/comb.json" ,"r") as f:
    a = json.loads(f.read())
with open("./motConfig0506.json" ,"a") as f:
    f.write(json.dumps(a))

In [ ]:
# currentLocation = initLocation.copy()
# currentSpeed = initSpeed.copy()
# path = './motImg0312/'
# ct = 0
# for i in range(int(round(t/CALCTIME,0))):
#     print(ct)
#     dis = calDis(currentLocation)
#     clps = checkClps(dis,RADIUS)
#     border = checkBorder(currentLocation,RADIUS)
#     currentSpeed = switchPair(currentSpeed,clps)
#     currentSpeed = switchBorder(currentSpeed,border)
#     currentLocation = updateLocation(currentSpeed= currentSpeed,currentLocation= currentLocation,acc= -MIU*g,t= CALCTIME)
#     currentSpeed = updateSpeed(currentSpeed=currentSpeed,acc=-MIU*g,t=CALCTIME)
#     img = pointDraw(currentLocation,RADIUS)
#     savePath = path+'frame'+str(ct) +".png"
#     img.savePng(savePath)
#     ct+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


KeyboardInterrupt: 

In [29]:
import json
s = {'clp':loc}
d = json.dumps(s)
with open('temot0312-弹性碰撞.json','a') as f:
    f.write(d)